In [0]:
%fs ls dbfs:/public/healthfitness

path,name,size,modificationTime
dbfs:/public/healthfitness/equipment.csv,equipment.csv,128,1741116087000
dbfs:/public/healthfitness/equipment.json,equipment.json,0,1741041940000
dbfs:/public/healthfitness/gym_members_exercise_tracking.csv,gym_members_exercise_tracking.csv,65136,1741035026000
dbfs:/public/healthfitness/members_data.csv,members_data.csv,71398,1741036829000
dbfs:/public/healthfitness/memberships.csv,memberships.csv,45138,1741029544000
dbfs:/public/healthfitness/payments.csv,payments.csv,35298,1741029395000


In [0]:
members_df=spark.read.format('csv')\
    .option('header','true')\
        .option('inferSchema','true')\
            .load('dbfs:/public/healthfitness/members_data.csv')
members_df.show(5)

+---------+----------+---------+------+---+-----------+--------------------+------------+
|member_id|First Name|Last Name|Gender|Age|Weight (kg)|               Email|Phone Number|
+---------+----------+---------+------+---+-----------+--------------------+------------+
|        1|     James|   Miller|  Male| 59|         61|james.miller78@ex...|  2141545168|
|        2|      John|  Johnson|  Male| 24|         86|john.johnson36@ex...|  9215712962|
|        3|    Robert|    Davis|  Male| 43|         81|robert.davis29@ex...|  4678651643|
|        4|   Michael|    Brown|  Male| 29|        123|michael.brown100@...|  7608899920|
|        5|   William|    Brown|  Male| 57|         90|william.brown72@e...|  6287529260|
+---------+----------+---------+------+---+-----------+--------------------+------------+
only showing top 5 rows



In [0]:
membership_df=spark.read.format('csv')\
    .option('header','true')\
        .option('inferSchema','true')\
            .load('dbfs:/public/healthfitness/memberships.csv')
membership_df.show(5)

+-------------+---------+---------------+----------+----------+--------------+
|Membership ID|Member ID|Membership Type|Start Date|  End Date|Payment Status|
+-------------+---------+---------------+----------+----------+--------------+
|            1|        1|      Quarterly|2024-03-27|2024-06-25|          Paid|
|            2|        2|         Annual|2024-11-23|2025-11-23|          Paid|
|            3|        3|         Annual|2024-07-11|2025-07-11|       Pending|
|            4|        4|        Premium|2024-04-06|2025-04-06|          Paid|
|            5|        5|        Monthly|2024-04-22|2024-07-21|          Paid|
+-------------+---------+---------------+----------+----------+--------------+
only showing top 5 rows



In [0]:
memberships=members_df.join(membership_df, members_df['member_id']==membership_df['Member ID'])\
    .drop(membership_df['Member ID'])
memberships.show(5)

+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+
|member_id|First Name|Last Name|Gender|Age|Weight (kg)|               Email|Phone Number|Membership ID|Membership Type|Start Date|  End Date|Payment Status|
+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+
|        1|     James|   Miller|  Male| 59|         61|james.miller78@ex...|  2141545168|            1|      Quarterly|2024-03-27|2024-06-25|          Paid|
|        2|      John|  Johnson|  Male| 24|         86|john.johnson36@ex...|  9215712962|            2|         Annual|2024-11-23|2025-11-23|          Paid|
|        3|    Robert|    Davis|  Male| 43|         81|robert.davis29@ex...|  4678651643|            3|         Annual|2024-07-11|2025-07-11|       Pending|
|        4|   Michael|    Brown|  Male| 29|        123|mic

In [0]:
memberships.count()

1000

In [0]:
payments=spark.read.format('csv')\
    .option('header','true')\
        .option('inferSchema','true')\
            .load('dbfs:/public/healthfitness/payments.csv')
payments.show(5)

+----------+---------+-----------+--------+--------------+------------+
|Payment ID|Member ID|Amount Paid|Currency|Payment Method|Payment Date|
+----------+---------+-----------+--------+--------------+------------+
|         1|        1|     357.62|     USD|        Online|  2024-12-03|
|         2|        2|     191.95|     INR|          Card|  2024-09-04|
|         3|        3|     283.78|     EUR|          Card|  2024-11-13|
|         4|        4|       50.5|     INR|          Card|  2024-04-13|
|         5|        5|     243.41|     CAD|        Online|  2024-03-19|
+----------+---------+-----------+--------+--------------+------------+
only showing top 5 rows



In [0]:
members_payments=memberships.join(payments, memberships['member_id']==payments['Member ID'])\
    .drop(payments['Payment Method'])\
        .drop(payments['Amount Paid'])\
            .drop(payments['Member ID'])\
                .drop(payments['Payment ID'])
members_payments.show(5)

+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+--------+------------+
|member_id|First Name|Last Name|Gender|Age|Weight (kg)|               Email|Phone Number|Membership ID|Membership Type|Start Date|  End Date|Payment Status|Currency|Payment Date|
+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+--------+------------+
|        1|     James|   Miller|  Male| 59|         61|james.miller78@ex...|  2141545168|            1|      Quarterly|2024-03-27|2024-06-25|          Paid|     USD|  2024-12-03|
|        2|      John|  Johnson|  Male| 24|         86|john.johnson36@ex...|  9215712962|            2|         Annual|2024-11-23|2025-11-23|          Paid|     INR|  2024-09-04|
|        3|    Robert|    Davis|  Male| 43|         81|robert.davis29@ex...|  4678651643|            3|  

In [0]:
members_payments.count()

1000

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
members_payments.select(col('currency')).distinct().show()

+--------+
|currency|
+--------+
|     GBP|
|     CAD|
|     EUR|
|     AUD|
|     INR|
|     USD|
+--------+



In [0]:
new_members_payments=members_payments.withColumn("country", when(col("currency")=='INR',"India")\
    .when(col("currency")=="USD","USA")\
        .when(col("currency")=="EUR","Europe")\
            .when(col("currency")=="GBP","UK")\
                .when(col("currency")=="CAD","Canada")\
                    .when(col("currency")=="AUD","Australia")\
                        .otherwise("Other"))

In [0]:
new_members_payments.show(5)

+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+--------+------------+-------+
|member_id|First Name|Last Name|Gender|Age|Weight (kg)|               Email|Phone Number|Membership ID|Membership Type|Start Date|  End Date|Payment Status|Currency|Payment Date|country|
+---------+----------+---------+------+---+-----------+--------------------+------------+-------------+---------------+----------+----------+--------------+--------+------------+-------+
|        1|     James|   Miller|  Male| 59|         61|james.miller78@ex...|  2141545168|            1|      Quarterly|2024-03-27|2024-06-25|          Paid|     USD|  2024-12-03|    USA|
|        2|      John|  Johnson|  Male| 24|         86|john.johnson36@ex...|  9215712962|            2|         Annual|2024-11-23|2025-11-23|          Paid|     INR|  2024-09-04|  India|
|        3|    Robert|    Davis|  Male| 43|         81|robert.dav

In [0]:
equipments_df=spark.read.format('csv')\
    .option('header','false')\
        .option('inferSchema','true')\
            .load('dbfs:/public/healthfitness/equipment.csv')\
                .withColumnRenamed('_c0','Equipment')\
                .withColumn("type",when(col("Equipment")=="barbell","Power Lifting")\
                    .when(col("Equipment")=="dumbbell","Strength")\
                        .when(col("Equipment")=="other","Yoga")\
                           .when(col("Equipment")=="body_only","Cardio")\
                               .when(col("Equipment")=="cable","IsolationExercises")\
                                   .when(col("Equipment")=="machine","Rehabilitation")\
                                       .when(col("Equipment")=="kettlebells","HIIT")\
                                           .when(col("Equipment")=="bands","Mobility")\
                                               .when(col("Equipment")=="medicine_ball","CoreStability")\
                                                   .when(col("Equipment")=="exercise_ball","BalanceTraining")\
                                                       .when(col("Equipment")=="foam_roll","Recovery")\
                                                           .when(col("Equipment")=="e-z_curl_bar","ArmDevelopment") )
equipments_df.show()

+-------------+------------------+
|    Equipment|              type|
+-------------+------------------+
|      barbell|     Power Lifting|
|     dumbbell|          Strength|
|        other|              Yoga|
|    body_only|            Cardio|
|        cable|IsolationExercises|
|      machine|    Rehabilitation|
|  kettlebells|              HIIT|
|        bands|          Mobility|
|medicine_ball|     CoreStability|
|exercise_ball|   BalanceTraining|
|    foam_roll|          Recovery|
| e-z_curl_bar|    ArmDevelopment|
+-------------+------------------+



In [0]:
gym_df=spark.read.format('csv')\
  .option('header','true')\
    .option('inferSchema','true')\
       .load('dbfs:/public/healthfitness/gym_members_exercise_tracking.csv')
gym_df=gym_df.drop(gym_df['Age'])\
  .drop(gym_df['Gender'])\
  .select("*",round(col('Weight (kg)')).cast("int").alias('Weight'))
gym_df=gym_df.drop(gym_df['Weight (kg)'])
gym_df.show(5)

+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+------+
|Height (m)|Max_BPM|Avg_BPM|Resting_BPM|Session_Duration (hours)|Calories_Burned|Workout_Type|Fat_Percentage|Water_Intake (liters)|Workout_Frequency (days/week)|Experience_Level|  BMI|Weight|
+----------+-------+-------+-----------+------------------------+---------------+------------+--------------+---------------------+-----------------------------+----------------+-----+------+
|      1.71|    180|    157|         60|                    1.69|         1313.0|        Yoga|          12.6|                  3.5|                            4|               3| 30.2|    88|
|      1.53|    179|    151|         66|                     1.3|          883.0|        HIIT|          33.9|                  2.1|                            4|               2| 32.0|    75|
|      1.66|    167|    122|         54|

In [0]:
gym_members_df=new_members_payments.join(gym_df,new_members_payments['Weight (kg)']==gym_df['Weight'])\
    .drop(new_members_payments['Weight (kg)'])\
    .drop(gym_df['Max_BPM'])\
        .drop(gym_df['Avg_BPM'])\
            .drop(gym_df['Resting_BPM'])\
            .withColumnRenamed('First Name','First_Name')\
                .withColumnRenamed('Last Name','Last_Name')\
                    .withColumnRenamed('Phone Number','Phone_Number')\
                        .withColumnRenamed('Membership ID','Membership_ID')\
                            .withColumnRenamed('Payment Status','Payment_Status')\
                                .withColumnRenamed('Membership Type','Membership_Type')\
                                    .withColumnRenamed('Start Date','Start_Date')\
                                        .withColumnRenamed('End Date','End_Date')\
                                            .withColumnRenamed('Payment Date','Payment_Date')\
                                            .withColumnRenamed('Height (m)','Height')\
                                                .withColumnRenamed('Session_Duration (hours)','Session_Duration')\
                                                    .withColumnRenamed('Water_Intake (liters)','Water_Intake')\
                                                        .withColumnRenamed('Workout_Frequency (days/week)','Workout_Frequency')
gym_members_df.count()

10852

In [0]:
gym_members_df.select("*").distinct().count()

10852

In [0]:
%sql
create database if not exists healthfitness_db;

In [0]:
%sql
use database healthfitness_db;

In [0]:
%sql
show tables

database,tableName,isTemporary
,_sqldf,true


In [0]:
final_df=gym_members_df.join(equipments_df,gym_members_df['Workout_Type']==equipments_df['type'])\
    .drop(equipments_df['type'])
final_df=final_df.select("*")
final_df.write.mode("overwrite").saveAsTable("gym_members_final")
final_df.write.mode("overwrite").parquet("dbfs:/public/healthfitness/gym_members_final")

In [0]:
%sql 
show tables

database,tableName,isTemporary
healthfitness_db,gym_members_final,false
,_sqldf,true


In [0]:
%fs ls dbfs:/public/healthfitness/gym_members_final/

path,name,size,modificationTime
dbfs:/public/healthfitness/gym_members_final/_SUCCESS,_SUCCESS,0,1741125009000
dbfs:/public/healthfitness/gym_members_final/_committed_465490544831604262,_committed_465490544831604262,123,1741125009000
dbfs:/public/healthfitness/gym_members_final/_started_465490544831604262,_started_465490544831604262,0,1741125009000
dbfs:/public/healthfitness/gym_members_final/part-00000-tid-465490544831604262-60459828-ce49-4cf4-8757-ce6d32f00078-348-1.c000.snappy.parquet,part-00000-tid-465490544831604262-60459828-ce49-4cf4-8757-ce6d32f00078-348-1.c000.snappy.parquet,226466,1741125009000
